In [ ]:
import os 
import sys 
import logging 

logging.basicConfig(level=logging.INFO)

cur_path = os.path.abspath(".")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy
from python_graphql_client import GraphqlClient

from utils.utils import remove_prefix, ddf, load_subgraph
from utils.vega import output_chart
sg, bs = load_subgraph()
# client = GraphqlClient(endpoint=url)
# df_ftokens = df_fert_tokens_raw.copy()

# nearest = alt.selection_single(
#     nearest=True, encodings=["x"], on="mouseover", empty="none"
# )
# brush = alt.selection_interval(encodings=['x'])

# base = alt.Chart(df_ftokens)
# x = alt.X("humidity:Q")
# y = alt.Y("fertilizer:Q")

# # Histogram of fertilizer purchased by season 
# histogram = base.mark_bar().encode(
#     x=x, y=y
# )

# # Vertical line to indicate nearest histogram bin 
# vline = base.mark_rule(color="grey").encode(
#     x=x, size=alt.value(1)
# ).transform_filter(
#     nearest
# )

# # Text label to indicate value nearest histogram bin 
# text = histogram.mark_text(align='left', dx=5, dy=-5).encode(
#     text=alt.condition(nearest, y, alt.value(' '))
# )

# # Time axis which drives scale of histogram 
# time_axis = base.mark_bar().encode(
#     x=x, y=y
# ).add_selection(
#     brush 
# ).resolve_scale(
#     'independent'
# ).properties(height=50)

# (
#     alt.layer(
#         histogram.add_selection(nearest), 
#         text, 
#         vline
#     ).transform_filter(
#         brush
#     ).properties(height=250) 
#     & time_axis 
# ) 
seasons = bs.Query.seasons(
    first=10000, where={"season_gte": 6074}, orderBy="season", orderDirection="asc"
)
df_seasons_raw = sg.query_df([
    seasons.season, 
    seasons.timestamp, 
], pagination_strategy=ShallowStrategy)
df_seasons_raw = remove_prefix(df_seasons_raw, 'seasons_')
print(len(df_seasons_raw))
df_seasons_raw.head()
# bs.FertilizerToken.sprouts = (
#     bs.FertilizerToken.supply * (bs.FertilizerToken.humidity + 100) / 100
# ) 
bs.FertilizerToken.fert = bs.FertilizerToken.supply 
bs.FertilizerToken.endBpf = SyntheticField(
  lambda _id: float(_id) / 1e6, 
  SyntheticField.FLOAT,
  bs.FertilizerToken.id, 
)
fert_tokens = bs.Query.fertilizerTokens(
    first=10000, 
    orderBy="humidity", 
    orderDirection="desc"
)
df_fert_tokens_raw = sg.query_df(
    [
        fert_tokens.season, 
        fert_tokens.fert, 
        fert_tokens.humidity, 
        fert_tokens.startBpf, 
        fert_tokens.endBpf, 
    ],
    pagination_strategy=ShallowStrategy
)
df_fert_tokens_raw = remove_prefix(df_fert_tokens_raw, "fertilizerTokens_")
df_fert_tokens_raw['startBpf'] /= 1e6
print(len(df_fert_tokens_raw))
df_fert_tokens_raw.head(10)

rewards = bs.Query.rewards(orderBy="blockNumber", orderDirection="asc", first=10000)

df_rewards_raw = sg.query_df([
    rewards.season, 
    rewards.toFertilizer, 
], pagination_strategy=ShallowStrategy)
df_rewards_raw = remove_prefix(df_rewards_raw, 'rewards_')
df_rewards_raw['toFertilizer'] = df_rewards_raw['toFertilizer'].astype(float) / 1e6 
df_rewards_raw.head()
df = df_seasons_raw.merge(
    df_rewards_raw, how="left", on="season"
).merge(
    df_fert_tokens_raw, how="left", on="season"
)
df = df.rename(columns={"toFertilizer": "to_fertilizer"})
df['timestamp'] = pd.to_datetime(df.timestamp, unit="s")

df['to_fertilizer'] = df.to_fertilizer.fillna(0)
df['fertilized_beans'] = df.to_fertilizer.cumsum()
df['unfertilized_beans'] = ((df.fert * df.endBpf).cumsum() - df.fertilized_beans).ffill()
df['sprouts_rinsable'] = df.fertilized_beans
df['sprouts'] = df.unfertilized_beans
df['rinsable_percent'] = df.fertilized_beans / (df.fertilized_beans + df.unfertilized_beans).ffill()

df['fertilizer_total'] = 77000000
df['fertilizer_active'] = df.fert.cumsum().ffill()
df['fertilizer_available'] = (df.fertilizer_total - df.fertilizer_active).ffill()

# ddf(df)
df = df.sort_values("timestamp")
df = df.loc[df.season >= 6075]
df.tail(5)
def visualize_recap_paid_percent(df: pd.DataFrame): 
    """"""
    df = df.iloc[1:,]
    # fertilizer breakdown 
    series_fert = df.iloc[-1,][['fertilizer_available', 'fertilizer_active']]
    fert_total = series_fert['fertilizer_available'] + series_fert['fertilizer_active']
    df_fert = pd.DataFrame(
        {
            'category': series_fert.index, 'value': series_fert.values, 
            'label': [
                f"{c}: {int(v):,} \n({v / fert_total:.1%})" 
                for c, v in zip(series_fert.index, series_fert.values)
            ]
        }
    )
    fertililzer_breakdown = alt.Chart(
        df_fert, title="Fertilizer Breakdown"
    ).encode(
        theta=alt.Theta("value:Q", stack=True),
        color=alt.Color("category:N", legend=alt.Legend()),
        tooltip="label:N"
    ).mark_arc(outerRadius=120, innerRadius=75)
    # sprouts breakdown 
    df_sprouts = df.copy()
    label_vars = ['sprouts rinsable', 'sprouts ', 'rinsable percent']
    id_vars = ['timestamp'] + label_vars
    value_vars = ['sprouts_rinsable', 'sprouts', 'rinsable_percent']
    df_sprouts['sprouts rinsable'] = df_sprouts['sprouts_rinsable']
    df_sprouts['sprouts '] = df_sprouts['sprouts']
    df_sprouts['rinsable percent'] = df_sprouts['rinsable_percent']
    df_sprouts = df_sprouts.melt(
        id_vars=id_vars, value_vars=value_vars
    ).sort_values("timestamp")
    sprouts_base = alt.Chart(
        df_sprouts[['timestamp', 'value', 'variable'] + label_vars], 
        title="Sprouts Breakdown"
    ).encode(
        x=alt.X("yearmonthdatehours(timestamp):T", axis=alt.Axis(tickCount=10, title="Date")), 
        y=alt.Y("value:Q", axis=alt.Axis(title="Sprouts")), 
        color=alt.Color("variable:N", legend=alt.Legend()),
    )
    sprouts_area = sprouts_base.transform_filter(
        datum.variable != 'rinsable_percent'
    ).mark_area(point="transparent").encode(
        tooltip=[
            alt.Tooltip(f'{e}:Q', format=",d" if e != 'rinsable percent' else ".2%") 
            for e in label_vars
        ]
    )
    sprouts_line = sprouts_base.transform_filter(
        datum.variable == 'rinsable_percent'
    ).mark_line().encode(
        y=alt.Y("value:Q", axis=alt.Axis(title="Rinsable Percent", format=".2%")), 
    )
    sprouts_breakdown = alt.layer(sprouts_area, sprouts_line).resolve_scale(
        y="independent", 
    ).resolve_axis(
        y="independent"
    ).properties(width=400)
    
    c = alt.hconcat(
        sprouts_breakdown, 
        fertililzer_breakdown
    ).resolve_legend(
        color="independent", 
    ).resolve_scale(
        y="independent",
        color="independent",
    ).resolve_axis(
        y="independent"
    )
    #c.save('../public/schemas/barn_breakdown.json')
    return c
 
c = visualize_recap_paid_percent(df)
c
output_chart(c)
